# Deploy R Shiny Dashboard App

### Introduction

Now that we have prepared the data and stored it, we will now:

* Create a deployment space. 
* Store Shiny dashboard assets into the deployment space.
* Deploy Shiny assets as an app and view the dashboard.

Watson Machine Learning provides deployment spaces where the user can save, configure and deploy the shiny assets. The shiny app can be deployed in a similar way to models and functions, using the [ibm-watson-machine-learning](http://ibm-wml-api-pyclient.mybluemix.net) package.




All of the files associated with the dashboard are contained in a zip file which is stored in data assets. CSV files stored in the zip file were generated from the `synthetic` data source in ***Data-Preparation*** notebook. If the user would like to make changes to the dashboard, they can download the zip from data assets and upload it in the RStudio IDE. 

### User Inputs

The user can specify the name for the space, R-shiny app deployments.
R-shiny app deployment will be stored in the deployment space. The deployment space can be created programmatically using `ibm-watson-machine-learning client`.
If a space with specified space_name already exists, user can either use the existing space by specifying `use_existing_space=True` or delete the existing space and create a new one by specifying `use_existing_space=False` below. By default use_existing_space is set to `True`.

In [2]:
# Specify a name for the space being created. 
space_name = 'Healthcare Services Location Optimization Space'

# Specify a name for the r shiny app being deployed. 
r_shiny_deployment_name='Healthcare_Services_Location_Optimization_Shiny_App'


use_existing_space=True

In [4]:

from ibm_watson_machine_learning import APIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.5"
}

client = APIClient(wml_credentials)

### Create the Deployment Space

Create a new deployment space using name of the space as specified in the user inputs cell above. 
By default, if the space with specified name already exists, the existing space will be used to store and deploy the app.

In [5]:
space_uid=""
for space in client.spaces.get_details()['resources']:

    if space['entity']['name'] ==space_name:
        print("Deployment space with ",space_name,"already exists . .")
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        if(use_existing_space==False):

            for deployment in client.deployments.get_details()['resources']:
                print("Deleting deployment",deployment['entity']['name'], "in the space",)
                deployment_id=deployment['metadata']['id']
                client.deployments.delete(deployment_id)
            print("Deleting Space ",space_name,)
            client.spaces.delete(space_uid)
            time.sleep(10)
        else:
            print("Using the existing space")
            
            
if (space_uid=="" or use_existing_space==False):
    print("\nCreating a new deployment space -",space_name)
    # create the space and set it as default
    space_meta_data = {
        client.spaces.ConfigurationMetaNames.NAME : space_name

        }

    stored_space_details = client.spaces.store(space_meta_data)

    space_uid = stored_space_details['metadata']['id']

    client.set.default_space(space_uid)


Creating a new deployment space - Healthcare Services Location Optimization Space
Space has been created. However some background setup activities might still be on-going. Check for 'status' field in the response. It has to show 'active' before space can be used. If its not 'active', you can monitor the state with a call to spaces.get_details(space_id)


### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [6]:
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "Healthcare_Services_Location_Optimization_Shiny_assets",
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space' # optional
}
app_details = client.shiny.store(meta_props, '/project_data/data_asset/healthcare_services_location_optimization-dashboard.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are 1) anyone with the link 2) Authenticated users 3) Collaborators in this deployment space

In [7]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy Healthcare_Services_Location_Optimization dashboard',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1}
}

# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: '6576aaa1-9449-4ed8-b327-6d103f6bf7b9' started

#######################################################################################


initializing........
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='f2115289-1998-4665-9e75-5ba9519a4a72'
------------------------------------------------------------------------------------------------




### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, select ***Deployments -> Spaces -> Healthcare Services Location Optimization Space -> Deployments -> Healthcare_Services_Location_Optimization_Shiny_App** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [8]:
print("{HOSTNAME}"+"/ml/v4/deployments/"+rshiny_deployment['metadata']['id'] + '/r_shiny')

{HOSTNAME}/ml/v4/deployments/f2115289-1998-4665-9e75-5ba9519a4a72/r_shiny
